In [1]:
import spacy
from spacy import displacy

import pandas as pd


nlp = spacy.load("../models/pt_core_news_sm_addresses")

In [4]:
text = (
    "Além do território continental, o Brasil também possui alguns grandes grupos de ilhas no oceano Atlântico como os Penedos de São Pedro e São Paulo, Fernando de Noronha (territórios estaduais de Pernambuco), Trindade e Martim Vaz, no Espírito Santo, e um complexo de pequenas ilhas e corais chamado Atol das Rocas (pertencente ao estado do Rio Grande do Norte). "
    "AV. Maria Quitéria, 841 - Brasília Feira de Santana. Bahia. Cep:44.088-078"
)

Trecho retirado da [Wikipedia](https://pt.wikipedia.org/wiki/Lista_dos_munic%C3%ADpios_mais_populosos_do_Brasil_por_estado).

Antes de explorar, vamos precisar da lista de cidades (retirada da Rede SUAS).

In [5]:
doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

In [27]:
# ConcatUF+Mun;IBGE;IBGE7;UF;Município;Região;População 2010;Porte;Capital;

cities = pd.read_csv('../data/cities/Lista_Municípios_com_IBGE_Brasil_Versao_CSV.csv', sep=";", encoding="latin-1")
cities

,ConcatUF+Mun,IBGE,IBGE7,UF,Município,Região,População 2010,Porte,Capital,Unnamed: 9
0,ROAlta Floresta D'oeste,110001,1100015,RO,Alta Floresta D´oeste,Região Norte,24392.0,Pequeno II,NaN,PROCV(A2;'[Lista de Municípios com IBGE - Bras...
1,ROAriquemes,110002,1100023,RO,Ariquemes,Região Norte,90353.0,Médio,NaN,PROCV(A2;'Lista de Municípios IBGE'!$A$2:$G$55...
2,ROCabixi,110003,1100031,RO,Cabixi,Região Norte,6313.0,Pequeno I,NaN,NaN
3,ROCacoal,110004,1100049,RO,Cacoal,Região Norte,78574.0,Médio,NaN,NaN
4,ROCerejeiras,110005,1100056,RO,Cerejeiras,Região Norte,17029.0,Pequeno I,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
5565,GOVianópolis,522200,5222005,GO,Vianópolis,Região Centro-Oeste,12548.0,Pequeno I,NaN,NaN
5566,GOVicentinópolis,522205,5222054,GO,Vicentinópolis,Região Centro-Oeste,7371.0,Pequeno I,NaN,NaN
5567,GOVila Boa,522220,5222203,GO,Vila Boa,Região Centro-Oeste,4735.0,Pequeno I,NaN,NaN
5568,GOVila Propício,522230,5222302,GO,Vila Propício,Região Centro-Oeste,5145.0,Pequeno I,NaN,NaN


In [28]:
cities["Município"].to_csv("../data/cities/Lista_Municípios_com_IBGE_Brasil_Versao_CSV_apenas_cidades.csv", index=False)

Vamos usar o [Rule-based Matching_](https://spacy.io/usage/rule-based-matching#entityruler) do Spacy para adicionar a lista de cidades ao nosso modelo. Podemos colocar no [padrão JSONL](https://spacy.io/usage/rule-based-matching#entityruler-files)
para facilitar a leitura.

Padrão esperado:

```
{"label": "CITY", "pattern": "Feira de Santana"}
```


In [29]:
# cities.jsonl
import json


with open("../data/cities/cities.jsonl", "w") as cities_jsonl:
    for city in cities["Município"]:
        cities_jsonl.write(json.dumps({"label": "CITY", "pattern": city}))
        cities_jsonl.write("\n")
cities_jsonl.close()